In [166]:
import pandas as pd
import numpy as np
# pd.set_option("display.max_columns", None)

In [168]:
# Read data files
psproject = pd.read_excel("pow_sharing_data/PSProject_NewDataset.xlsx")
dpi2017 = pd.read_excel("pow_sharing_data/DPI2017_basefile_Jan2018.xlsx")
wgi_gov_effect = pd.read_excel("pow_sharing_data/Modified_WGIdataset copy.xlsx", sheet_name='GovernmentEffectiveness')
wgi_reg_quality = pd.read_excel("pow_sharing_data/Modified_WGIdataset copy.xlsx", sheet_name='RegulatoryQuality')
wgi_corruption = pd.read_excel("pow_sharing_data/Modified_WGIdataset copy.xlsx", sheet_name='ControlofCorruption')
v_dem = pd.read_excel("pow_sharing_data/V-Dem_Vars_tomerge.xlsx")
world_dev_all = pd.read_csv("pow_sharing_data/WorldDev_Ind_AllCountries.csv")

In [183]:
# display(psproject.head())
# display(dpi2017.head())
# display(wgi_gov_effect.head())
# display(world_dev_all.head())
# display(v_dem.head())

In [170]:
# Remove Unnamed columns from dpi2017
dpi2017 = dpi2017.loc[:, ~dpi2017.columns.str.contains('^Unnamed')]
# Rename country names column of the all datasets
dpi2017.rename(columns={"countryname": "cname"},inplace=True)
v_dem.rename(columns={"country_name": "cname"},inplace=True)
world_dev_all.rename(columns={"Country Name": "cname"},inplace=True)
world_dev_all.rename(columns={"Series Name": "series_name"},inplace=True)
wgi_gov_effect.rename(columns={"Country/Territory": "cname"}, inplace=True)
wgi_reg_quality.rename(columns={"Country/Territory": "cname"}, inplace=True)
wgi_corruption.rename(columns={"Country/Territory": "cname"}, inplace=True)

# Lowercase the country names
psproject['cname'] = psproject['cname'].str.lower()
dpi2017['cname'] = dpi2017['cname'].str.lower()
world_dev_all['cname'] = world_dev_all['cname'].str.lower()
v_dem['cname'] = v_dem['cname'].str.lower()
wgi_gov_effect['cname'] = wgi_gov_effect['cname'].str.lower()
wgi_reg_quality['cname'] = wgi_reg_quality['cname'].str.lower()
wgi_corruption['cname'] = wgi_corruption['cname'].str.lower()

In [171]:
# Fix country names based on psproject dataset
psproject.loc[(psproject.cname == 'congo, democratic republic'), 'cname'] = 'democratic republic of congo'

dpi2017.loc[(dpi2017.cname == 'bosnia-herz'),'cname'] = 'bosnia and herzegovina'
dpi2017.loc[(dpi2017.cname == 'congo (drc)'), 'cname'] = 'democratic republic of congo'
dpi2017.loc[(dpi2017.cname == 'dom. rep.'), 'cname'] = 'dominican republic'
dpi2017.loc[(dpi2017.cname == 'p. n. guinea'), 'cname'] = 'papua new guinea'
dpi2017.loc[(dpi2017.cname == 's. africa'), 'cname'] = 'south africa'
dpi2017.loc[(dpi2017.cname == 'yemen (ar)'), 'cname'] = 'yemen, north'
dpi2017.loc[(dpi2017.cname == 'yemen (pdr)'), 'cname'] = 'yemen, south'

v_dem.loc[(v_dem.cname == 'democratic republic of the congo'), 'cname'] = 'democratic republic of congo'
v_dem.loc[(v_dem.cname == 'ivory coast'), 'cname'] = "cote d'ivoire"
v_dem.loc[(v_dem.cname == 'south yemen'), 'cname'] = 'yemen, south'
v_dem.loc[(v_dem.cname == 'yemen') & (v_dem.year < 1990), 'cname'] = 'yemen, north'


world_dev_all.loc[(world_dev_all.cname == 'congo, dem. rep.'), 'cname'] = 'democratic republic of congo'
world_dev_all.loc[(world_dev_all.cname == 'congo, rep.'), 'cname'] = 'congo'
world_dev_all.loc[(world_dev_all.cname == 'lao pdr'), 'cname'] = 'laos'
world_dev_all.loc[(world_dev_all.cname == 'yemen, rep.'), 'cname'] = 'yemen, north'

# Fix country names of the all sheets of wgi dataset
for dataset in [wgi_gov_effect, wgi_reg_quality, wgi_corruption]:
    dataset.loc[(dataset.cname == 'congo, dem. rep.'), 'cname'] = 'democratic republic of congo'
    dataset.loc[(dataset.cname == 'congo, rep.'), 'cname'] = 'congo'
    dataset.loc[(dataset.cname == 'yemen, rep.'), 'cname'] = 'yemen'
    dataset.loc[(dataset.cname == "côte d'ivoire"), 'cname'] = "cote d'ivoire"
    dataset.loc[(dataset.cname == "lao pdr"), 'cname'] = "laos"

In [172]:
# Convert data types to make them ready to merge
psproject = psproject.fillna(-999)
psproject.cname = psproject.cname.astype('str')
psproject.year = psproject.year.astype('int64')

psproject_converted = psproject.loc[:, ~psproject.columns.isin(['year', 'cname', 'chga_demo'])].astype(float)
psproject = pd.concat([psproject[['year', 'cname', 'chga_demo']], psproject_converted], axis=1).reset_index(drop=True)

dpi2017 = dpi2017.fillna(-999)
dpi2017_converted = dpi2017.loc[:, ~dpi2017.columns.isin(['year', 'cname', 'execme', 
                                                          'ifs', 'gov1me', 'gov2me', 
                                                          'gov3me', 'opp1me', 'opp2me', 
                                                          'opp3me', 'nonchief'])].astype(float)
dpi2017 = pd.concat([dpi2017[['year', 'cname', 'execme', 'ifs', 
                              'gov1me', 'gov2me', 'gov3me', 
                              'opp1me', 'opp2me', 'opp3me', 
                              'nonchief']], dpi2017_converted], axis=1).reset_index(drop=True)

v_dem = v_dem.fillna(-999)
v_dem_converted = v_dem.loc[:, ~v_dem.columns.isin(['year', 'cname'])].astype(float)
v_dem = pd.concat([v_dem[['year', 'cname']], v_dem_converted], axis=1).reset_index(drop=True)


In [173]:
# Convert wgi dataset from wide to long format
wgi_gov_effect = wgi_gov_effect.melt(id_vars=['cname'], 
                                     value_vars=wgi_gov_effect.columns[2:])
wgi_reg_quality = wgi_reg_quality.melt(id_vars=['cname'], 
                                     value_vars=wgi_reg_quality.columns[2:])
wgi_corruption = wgi_corruption.melt(id_vars=['cname'], 
                                     value_vars=wgi_corruption.columns[2:])

# Change the auto-assigned column name back to year
for dataset in [wgi_gov_effect, wgi_reg_quality, wgi_corruption]:
    dataset.rename(columns={'variable': 'year'}, inplace=True)

In [174]:
# First merge the first two sheets of wgi data based on cname and year. 
# Then merge it with the third sheet
merged_wgi1 = pd.merge(wgi_gov_effect, wgi_reg_quality, on=["cname", "year"])
merged_wgi = pd.merge(merged_wgi1, wgi_corruption, on=["cname", "year"])

# Name the columns of the data
merged_wgi.columns = ['cname', 'year', 'government_effectiveness', 
                'regulatory_quality', 'control_of_corruption']

display(merged_wgi.head())

,cname,year,government_effectiveness,regulatory_quality,control_of_corruption
0,aruba,1996,NaN,NaN,NaN
1,andorra,1996,1.414038,1.266440,1.318143
2,afghanistan,1996,-2.175167,-2.090330,-1.291705
3,angola,1996,-0.859940,-1.415345,-1.167702
4,anguilla,1996,NaN,NaN,NaN


In [175]:
# Remove the unnecessary strings after the year from the columns
world_dev_all = world_dev_all.rename(columns=lambda x: x.split()[0])
# Convert dataset from wide to long format
world_dev_all = world_dev_all.melt(id_vars=['cname', 'series_name'], 
                                   value_vars=world_dev_all.columns[4:])
# Change the auto-assigned column name back to year
world_dev_all.rename(columns={'variable': 'year', 'series_name':'variables'}, inplace=True)
display(world_dev_all.head())

,cname,variables,year,value
0,afghanistan,Electric power consumption (kWh per capita),1960,NaN
1,afghanistan,"Literacy rate, adult total (% of people ages 1...",1960,NaN
2,afghanistan,Logistics performance index: Quality of trade ...,1960,NaN
3,afghanistan,Current health expenditure (% of GDP),1960,NaN
4,afghanistan,"Taxes on income, profits and capital gains (% ...",1960,NaN


In [176]:
# Convert the shape of the dateset to the same shape of the other datasets. 
world_dev_all_cleaned = world_dev_all.groupby(['cname','variables', 'year'])['value'].sum().unstack(-1).T.stack('cname').reset_index()
world_dev_all_cleaned = world_dev_all_cleaned.replace(0,np.nan)


In [177]:
world_dev_all_ = pd.DataFrame()
world_dev_all_['cname'] = world_dev_all_cleaned.cname.reset_index(drop=True)
world_dev_all_['year'] = world_dev_all_cleaned.year.reset_index(drop=True)
world_dev_all_['Current health expenditure (% of GDP)'] = world_dev_all_cleaned['Current health expenditure (% of GDP)'].reset_index(drop=True)
world_dev_all_['Electric power consumption (kWh per capita)'] = world_dev_all_cleaned['Electric power consumption (kWh per capita)'].reset_index(drop=True)
world_dev_all_['Literacy rate, adult total (% of people ages 15 and above)'] = world_dev_all_cleaned['Literacy rate, adult total (% of people ages 15 and above)'].reset_index(drop=True)
world_dev_all_['Logistics performance index: Quality of trade and transport-related infrastructure (1=low to 5=high)'] = world_dev_all_cleaned['Logistics performance index: Quality of trade and transport-related infrastructure (1=low to 5=high)'].reset_index(drop=True)
world_dev_all_['Taxes on income, profits and capital gains (% of total taxes)'] = world_dev_all_cleaned['Taxes on income, profits and capital gains (% of total taxes)'].reset_index(drop=True)

world_dev_all_ = world_dev_all_.fillna(-999)
world_dev_all_.cname = world_dev_all_.cname.astype('str')
world_dev_all_.year = world_dev_all_.year.astype('int64')
world_dev_all_converted = world_dev_all_.loc[:, ~world_dev_all_.columns.isin(['year', 'cname'])].astype(float)
world_dev_all_ = pd.concat([world_dev_all_[['year', 'cname']], world_dev_all_converted], axis=1).reset_index(drop=True)

world_dev_all_.loc[(world_dev_all_.cname == 'yemen, north') & (world_dev_all_.year >= 1990), 'cname'] = 'yemen'


Merging all datasets

In [178]:
ps_dpi_merged = pd.merge(psproject, dpi2017, how="left", on=["cname", "year"])

In [179]:
# Keep only related columns to reduce the size of the dataframe
ps_dpi_merged = ps_dpi_merged[
[ 'year',
  'cname',
'ten_years',
 'tsls2',
 'tsls3',
 'tsls',
 'ccode',
 'ccodecow',
 'al_ethnic',
 'al_language',
 'pwt_gsg',
 'chga_demo',
 'laglogpwt_rgdpch',
 'highps',
 'lnwardur',
 'numps',
 'newonsetdemulf',
 'pko',
 'mediation',
 'prevcivwar',
 'prevdem',
 'pcw',
 'rrs',
 'log_wdi_exp',
 'negsett',
 'milvictory',
 'polps',
 'milps',
 'econps',
 'terrps',
 'lji_region_year',
 'british_colonialism',
 'log_wdi_pop',
 'atwar',
 'lji',
 'log_rgdppc',
 'log_oda_per_capita_dollar150',
 'log_ngo',
 'y1999',
 'v2xeg_eqprotec_7v',
 'v2xeg_eqaccess_7v',
 'v2xeg_eqdr_7v',
 'v2xeg_eqaccess_reg_mean',
 'v2xeg_eqdr_reg_mean',
 'socialist_legcode',
 'v2x_poly_dif2',
 'v2x_poly_dif5',
 'v2x_poly_dif10',
 'v2x_libdem_dif2',
 'v2x_libdem_dif5',
 'v2x_libdem_dif10',
 'v2x_egal_dif2',
 'v2x_egal_dif5',
 'v2x_egal_dif10',
 'prewar_lib_dem_avg',
 'prewar_egal_dem_avg',
 'prewar_elec_dem_avg',
 'closedpr',
 'openpr',
 'coalitioninexecutive',
 'coalitioninlegislature',
 'propadmin',
 'muni',
 'state',
 'author']]

In [180]:
ps_dpi_vdem = pd.merge(ps_dpi_merged, v_dem, how="left", on=["cname", "year"])
ps_dpi_vdem_wgi = pd.merge(ps_dpi_vdem, merged_wgi, how="left", on=["cname", "year"])
all_merged = pd.merge(ps_dpi_vdem_wgi, world_dev_all_, how="left", on=["cname", "year"])

In [181]:
all_merged = all_merged.replace(-999, np.nan)

In [182]:
display(all_merged.head())

,year,cname,ten_years,tsls2,tsls3,tsls,ccode,ccodecow,al_ethnic,al_language,pwt_gsg,chga_demo,laglogpwt_rgdpch,highps,lnwardur,numps,newonsetdemulf,pko,mediation,prevcivwar,prevdem,pcw,rrs,log_wdi_exp,negsett,milvictory,polps,milps,econps,terrps,lji_region_year,british_colonialism,log_wdi_pop,atwar,lji,log_rgdppc,log_oda_per_capita_dollar150,log_ngo,y1999,v2xeg_eqprotec_7v,v2xeg_eqaccess_7v,v2xeg_eqdr_7v,v2xeg_eqaccess_reg_mean,v2xeg_eqdr_reg_mean,socialist_legcode,v2x_poly_dif2,v2x_poly_dif5,v2x_poly_dif10,v2x_libdem_dif2,v2x_libdem_dif5,v2x_libdem_dif10,v2x_egal_dif2,v2x_egal_dif5,v2x_egal_dif10,prewar_lib_dem_avg,prewar_egal_dem_avg,prewar_elec_dem_avg,closedpr,openpr,coalitioninexecutive,coalitioninlegislature,propadmin,muni,state,author,v2elffelr,v2elffelr_codelow,v2elffelr_codehigh,v2elffelr_sd,v2elffelr_osp,v2elffelr_osp_codelow,v2elffelr_osp_codehigh,v2elffelr_osp_sd,v2elffelr_ord,v2elffelr_ord_codelow,v2elffelr_ord_codehigh,v2elffelr_mean,v2elffelr_nr,v2elffelrbin,v2elffelrbin_codelow,v2elffelrbin_codehigh,v2elffelrbin_sd,v2elffelrbin_osp,v2elffelrbin_osp_codelow,v2elffelrbin_osp_codehigh,v2elffelrbin_osp_sd,v2elffelrbin_ord,v2elffelrbin_ord_codelow,v2elffelrbin_ord_codehigh,v2elffelrbin_mean,v2elffelrbin_nr,v2pssunpar,v2pssunpar_codelow,v2pssunpar_codehigh,v2pssunpar_sd,v2pssunpar_osp,v2pssunpar_osp_codelow,v2pssunpar_osp_codehigh,v2pssunpar_osp_sd,v2pssunpar_ord,v2pssunpar_ord_codelow,v2pssunpar_ord_codehigh,v2pssunpar_mean,v2pssunpar_nr,government_effectiveness,regulatory_quality,control_of_corruption,Current health expenditure (% of GDP),Electric power consumption (kWh per capita),"Literacy rate, adult total (% of people ages 15 and above)",Logistics performance index: Quality of trade and transport-related infrastructure (1=low to 5=high),"Taxes on income, profits and capital gains (% of total taxes)"
0,1946,afghanistan,NaN,NaN,NaN,NaN,4.0,700.0,0.769345,0.614146,NaN,0. Dictatorship,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,1.0,0.192910,0.265302,0.237746,0.220957,0.281155,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.731,-2.465,-1.086,0.712,0.7,0.196,1.078,0.455,1.0,0.0,1.0,0.667,3.0,-1.071,-1.501,-0.534,0.513,0.637,0.492,0.829,0.178,1.0,1.0,1.0,0.75,4.0,0.354,-0.338,1.038,0.709,1.209,0.844,1.651,0.389,1.0,0.0,1.0,1.25,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1947,afghanistan,NaN,NaN,NaN,NaN,4.0,700.0,0.769345,0.614146,NaN,0. Dictatorship,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,1.0,0.192910,0.265302,0.237746,0.262220,0.237627,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.731,-2.465,-1.086,0.712,0.7,0.196,1.078,0.455,1.0,0.0,1.0,0.667,3.0,-1.071,-1.501,-0.534,0.513,0.637,0.492,0.829,0.178,1.0,1.0,1.0,0.75,4.0,0.354,-0.338,1.038,0.709,1.209,0.844,1.651,0.389,1.0,0.0,1.0,1.25,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1948,afghanistan,NaN,NaN,NaN,NaN,4.0,700.0,0.769345,0.614146,NaN,0. Dictatorship,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.278083,0.0,NaN,0.0,0.0186,NaN,NaN,NaN,1.0,0.192910,0.265302,0.229674,0.297089,0.262806,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.731,-2.465,-1.086,0.712,0.7,0.196,1.078,0.455,1.0,0.0,1.0,0.667,3.0,-1.071,-1.501,-0.534,0.513,0.637,0.492,0.829,0.178,1.0,1.0,1.0,0.75,4.0,0.354,-0.338,1.038,0.709,1.209,0.844,1.651,0.389,1.0,0.0,1.0,1.25,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1949,afghanistan,NaN,NaN,NaN,NaN,4.0,700.0,0.769345,0.614146,NaN,0. Dictatorship,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.284267,0.0,NaN,0.0,0.0182,NaN,NaN,NaN,1.0,0.192910,0.265302,0.229674,0.295766,0.256027,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.731,-2.465,-1.086,0.712,0.7,0.196,1.078,0.455,1.0,0.0,1.0,0.667,3.0,-1.071,-1.501,-0.534,0.513,0.637,0.492,0.829,0.178,

In [165]:
all_merged.to_excel('power_sharing_merged_data.xlsx')